In [1]:
import warnings

import numpy as np
import pandas as pd

from catboost import CatBoostRegressor
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, Lasso

from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm import tqdm

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

## Reading datasets
Some notes have to be made here:
* The "revealed" dataset contains an extra training set that was released at an advanced stage of the competition.
* "Orders" is a dataset which was not provided by the competition host, but added by me. It includes data about the orders people were given to stay at home during covid, indicating how their degree of strictness.

In [2]:
train = pd.read_csv("../input/godaddy-microbusiness-density-forecasting/train.csv")
revealed = pd.read_csv("../input/godaddy-microbusiness-density-forecasting/revealed_test.csv")
train = pd.concat([train, revealed])
test = pd.read_csv("../input/godaddy-microbusiness-density-forecasting/test.csv")
census = pd.read_csv("../input/godaddy-microbusiness-density-forecasting/census_starter.csv", index_col="cfips")
coords = pd.read_csv("../input/usa-counties-coordinates/cfips_location.csv", index_col="cfips")
orders = pd.read_csv("../input/stay-at-home-orders/U.S._State_and_Territorial_Stay-At-Home_Orders__March_15__2020___May_31__2021_by_County_by_Day.csv").drop("Citations", axis=1)

In [3]:
# SMAPE implementation from user Giba
# This is the metric used to measure a competitor's performance
def smape(y_true, y_pred):
    smap = np.zeros(y_true.shape[0])
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.nanmean(smap)

## Basic features
We define time-based features, such as "year" or "month", plus some general ones, like the mean MBD across a particular state.

In [5]:
# Concat training and test sets to ease preprocessing
train["is_test"] = 0
test["is_test"] = 1

df_all = pd.concat([train, test]).sort_values(['cfips','row_id']).reset_index(drop=True)

# Define some features
df_all["first_day_of_month"] = pd.to_datetime(df_all["first_day_of_month"])
df_all["year"] = df_all["first_day_of_month"].dt.year
df_all["month"] = df_all["first_day_of_month"].dt.month
df_all["is_january"] = (df_all["month"] == 1)

df_all.drop(df_all[(df_all["is_test"] == 1) & (df_all["year"] == 2022) & (df_all["month"] > 10)].index, axis=0, inplace=True)

df_all['county'] = df_all.groupby('cfips')['county'].ffill()
df_all['state'] = df_all.groupby('cfips')['state'].ffill()
# Month count
df_all["dcount"] = df_all.groupby(['cfips'])['row_id'].cumcount()

df_all['county_i'] = (df_all['county'] + df_all['state']).factorize()[0]
df_all['state_i'] = df_all['state'].factorize()[0]

# We will set the target as the relative change in MBD. This could
# be a simpler objective for the final model.

# However, since having 0 as the MBD for the previous month would result in
# an undefined value (division by zero), we first replace those values by 1.
df_all['target'] = df_all.groupby('cfips')['microbusiness_density'].shift(1)
df_all["target"] = (df_all["target"] != 0) * df_all["target"] + (df_all["target"] == 0)
df_all['target'] = df_all['microbusiness_density'] / df_all['target'] - 1

counties = df_all["cfips"].unique()
states = df_all["state"].unique()
dcounts = df_all["dcount"].unique()

# Add covid stay-at-home orders, by the level of strictness (how much
# freedom people have to go out)
orders["Date"] = pd.to_datetime(orders["Date"])
#orders = orders.set_index("Date")
orders["year"] = orders["Date"].dt.year
orders["month"] = orders["Date"].dt.month

# Take the monthly mean of the level of strictness
for year in orders["year"].unique():
    print(year)
    yr_idx = df_all["year"] == year
    yr_idx_orders = orders["year"] == year
    for month in tqdm(orders.loc[orders["year"] == year, "month"].unique()):
        mn_idx = df_all["month"] == month
        mn_idx_orders = orders["month"] == month
        for county in counties:
            df_all.loc[yr_idx & mn_idx & (df_all["cfips"] == county), "order_level"] = 4 - orders.loc[yr_idx_orders & mn_idx_orders & (orders["FIPS_Code"] == county), "SAH_Order_Code"].mean()


for county in counties:
    # Add the county's coordinates
    df_all.loc[df_all["cfips"] == county, "lng"] = coords.loc[county, "lng"]
    df_all.loc[df_all["cfips"] == county, "lat"] = coords.loc[county, "lat"]
    # Compute a rolling mean of the level of strictness
    df_all.loc[df_all["cfips"] == county, "mean_level"] = df_all["order_level"].rolling(6).mean()
    df_all.loc[(df_all["cfips"] == county) & df_all["mean_level"].isnull(), "mean_level"] = 0

for state in states:
    for dc in dcounts:
        # Take the target's monthly mean and std for each state
        state_idx = (df_all["dcount"] == dc) & (df_all["state"] == state)
        s = df_all.loc[state_idx, "target"]
        df_all.loc[state_idx, "mean_state_tg"] = s.mean()
        df_all.loc[state_idx, "state_tg_std"] = s.std()
        
# District of Columbia has a single county, so std cannot be computed
df_all.loc[df_all["state"] == "District of Columbia", "state_tg_std"] = 0

# Split sets again
train = df_all.loc[df_all["is_test"] == 0].drop("is_test", axis=1)
test = df_all.loc[df_all["is_test"] == 1].drop("is_test", axis=1)

# Compute marginal target and marginal MBD (the true difference between
# a month's value and that from the previous month)
for county in counties:
    county_df = train[train["cfips"] == county]
    
    train.loc[train["cfips"] == county, "prev_marg_density"] = county_df["microbusiness_density"].shift(1) - county_df["microbusiness_density"].shift(2)
    train.loc[train["cfips"] == county, "prev_marg_target"] = county_df["target"].shift(1) - county_df["target"].shift(2)

2022


100%|██████████| 8/8 [05:57<00:00, 44.72s/it]


2021


100%|██████████| 12/12 [09:03<00:00, 45.26s/it]


2020


100%|██████████| 10/10 [07:19<00:00, 43.99s/it]


## Linear models and lags
We will fit a linear and a lasso regression for each count. Their predictions and their slopes will be used as extra features.

On the other hand, we will create MBD lags (features that reflect values from previous months) and moving averages.

In [14]:
model_names = ["LR", 
    #"ridge", 
    "lasso", 
    #"elasticnet"
]

sorted_train = train.sort_values("first_day_of_month")
sorted_test = test.sort_values("first_day_of_month")

In [16]:
# Using TimeSeriesSplit creates a split where an uneven number of rows
# per county is assigned to the indexes. Therefore, we have to create
# a custom time split.
def custom_split(df, n_splits):
    # custom_split assumes df is sorted by first_day_of_month
    county_size = df.groupby("cfips").size()
    if not (county_size.min() == county_size.max()):
        print("WARNING: Dataframe does not have an equal number of rows per county")
    n_months = len(df["first_day_of_month"].unique())
    split_len = int(n_months / (n_splits + 1)) * len(counties)
    
    last = split_len
    splits = []
    for i in range(1, n_splits + 1):
        if i == n_splits:
            splits.append((np.arange(0, last), np.arange(last, df.shape[0])))
            break
        splits.append((np.arange(0, last), np.arange(last, last + split_len)))
        last += split_len
        
    return splits

In [ ]:
train_index, valid_index = custom_split(sorted_train, 8)[-1]
nr_feats = list(sorted_train.columns)

st_dcount = sorted(sorted_train["dcount"].unique())
        
sub_train = sorted_train.iloc[train_index]
valid = sorted_train.iloc[valid_index]

cond_1 = (sub_train.groupby("cfips").size().min() == sub_train.groupby("cfips").size().max())
cond_2 = (valid.groupby("cfips").size().min() == valid.groupby("cfips").size().max())

if not (cond_1 and cond_2):
    print("WARNING: Subtraining and/or validation set do not have an equal number of rows per county")

# Initialize columns to nan
for name in model_names:
    sub_train.loc[:, name] = np.nan
    valid.loc[:, name] = np.nan
    sorted_test.loc[:, name] = np.nan
    
    nr_feats.append(name)
    nr_feats.append(name + "_slope")

# Anomaly detection (dismissed; read note below) and linear features
anom_rates = []
for county in counties:
    county_train_df = sub_train[sub_train["cfips"] == county]
    county_valid_df = valid[valid["cfips"] == county]
    ctd_shape = county_train_df.shape[0]
    cvd_shape = county_valid_df.shape[0]
    cttd_shape = test[test["cfips"] == county].shape[0]

    county_X_train = np.arange(ctd_shape).reshape(-1, 1)
    county_X_valid = np.arange(ctd_shape, ctd_shape + cvd_shape).reshape(-1, 1)
    county_X_test = np.arange(ctd_shape + cvd_shape, ctd_shape + cvd_shape + cttd_shape).reshape(-1, 1)

    # Detect anomalies in time series and replace them by the expected
    # value for the time step in question, i.e. trend + seasonal
    
    # NOTE: This piece of code has no effect over the final dataset. It
    # was used to test a preprocessing step, but was then dismissed.
    county_y_train = county_train_df["microbusiness_density"]
    if county_y_train.shape[0] >= 24:
        county_y_train.index = county_train_df["first_day_of_month"]
        sd = seasonal_decompose(county_y_train, model="additive", 
                                extrapolate_trend="freq")
        q1, q3 = np.percentile(county_y_train.sort_values(),[25,75])
        iqr = q3 - q1
        low = q1 - (1.5 * iqr)
        up = q3 + (1.5 * iqr) 
        anom = (county_y_train < low) | (county_y_train > up)
        # Multiply by boolean array, so that only anomalies change
        county_y_train = (1 - anom) * county_y_train + anom * (sd.seasonal + sd.trend)
        anom_rates.append(anom.sum() / anom.shape[0])

    # Fit a linear (standard and lasso) regression over the MBD county-wise.
    # We will feed the final model with the predictions from these regressions
    # and their slopes.
    models = [LinearRegression(), 
              #Ridge(), 
              Lasso(), 
              #ElasticNet(alpha=0.3)
    ]
    
    for model, name in zip(models, model_names):
        model.fit(county_X_train, county_y_train)

        train_preds = model.predict(county_X_train)
        valid_preds = model.predict(county_X_valid)
        test_preds = model.predict(county_X_test)
        
        # Add linear predictions
        sub_train.loc[sub_train["cfips"] == county, name] = train_preds
        valid.loc[valid["cfips"] == county, name] = valid_preds
        sorted_test.loc[sorted_test["cfips"] == county, name] = test_preds
        
        # Add slopes
        sub_train.loc[sub_train["cfips"] == county, name + "_slope"] = model.coef_[0]
        valid.loc[valid["cfips"] == county, name + "_slope"] = model.coef_[0]
        sorted_test.loc[sorted_test["cfips"] == county, name + "_slope"] = model.coef_[0]

In [18]:
# Non-recurrent features
sub_train["set"] = "train"
valid["set"] = "valid"
sorted_test["set"] = "test"
nr_all = pd.concat([sub_train, valid, sorted_test])[nr_feats + ["set"]]

# Create time features. This includes lags (MBD and target from the previous
# months) and moving averages with their std.
for county in tqdm(counties):
    # Add lag 6 separately so we do not compute mean and std for a window
    # of 1 (which would not have sense)
    nr_all.loc[nr_all["cfips"] == county, "mbd_lag_5"] = nr_all.loc[nr_all["cfips"] == county, "microbusiness_density"].shift(5)
    nr_all.loc[nr_all["cfips"] == county, "tg_lag_5"] = nr_all.loc[nr_all["cfips"] == county, "target"].shift(5)
    nr_all.loc[nr_all["cfips"] == county, "mean_state_tg_lag_5"] = nr_all.loc[nr_all["cfips"] == county, "mean_state_tg"].shift(5)
    nr_all.loc[nr_all["cfips"] == county, "state_tg_std_lag_5"] = nr_all.loc[nr_all["cfips"] == county, "state_tg_std"].shift(5)
    for j in range(2, 5):
        # Add MBD from j+4 months ago
        nr_all.loc[nr_all["cfips"] == county, "mbd_lag_" + str(j+4)] = nr_all.loc[nr_all["cfips"] == county, "microbusiness_density"].shift(j+4)
        # Add target from j+4 months ago
        nr_all.loc[nr_all["cfips"] == county, "tg_lag_" + str(j+4)] = nr_all.loc[nr_all["cfips"] == county, "target"].shift(j+4)
        # Add the state's target mean from j+4 months ago
        nr_all.loc[nr_all["cfips"] == county, "mean_state_tg_lag_" + str(j+4)] = nr_all.loc[nr_all["cfips"] == county, "mean_state_tg"].shift(j+4)
        # Add the state's target std from j+4 months ago
        nr_all.loc[nr_all["cfips"] == county, "state_tg_std_lag_" + str(j+4)] = nr_all.loc[nr_all["cfips"] == county, "state_tg_std"].shift(j+4)
        
        # Compute the MBD's rolling mean and std (window starts 5+j months ago 
        # and ends 5 months ago)
        s = nr_all.loc[nr_all["cfips"] == county, "mbd_lag_5"].rolling(window=j)
        nr_all.loc[nr_all["cfips"] == county, "MA_" + str(j)] = s.mean().shift(1)
        nr_all.loc[nr_all["cfips"] == county, "Mstd_" + str(j)] = s.std().shift(1)

        # Compute the target's rolling mean and std (same window)
        t = nr_all.loc[nr_all["cfips"] == county, "tg_lag_5"].rolling(window=j)
        nr_all.loc[nr_all["cfips"] == county, "t_MA_" + str(j)] = t.mean().shift(1)
        nr_all.loc[nr_all["cfips"] == county, "t_Mstd_" + str(j)] = t.std().shift(1)

        # Compute the rolling mean and std (same window) for the overall state's 
        # monthly target mean
        u = nr_all.loc[nr_all["cfips"] == county, "mean_state_tg_lag_5"].rolling(window=j)
        nr_all.loc[nr_all["cfips"] == county, "state_t_MA_" + str(j)] = u.mean().shift(1)
        nr_all.loc[nr_all["cfips"] == county, "state_t_Mstd_" + str(j)] = u.std().shift(1)

# Group counties in clusters
# NOTE: The resulting features did not have an important effect in the
# final model's performance. Instead, lag features and "linear" ones
# were the key.

cluster_df = nr_all[["lng", "lat", "mbd_lag_5"]].copy()

# Scale features before clustering
lat_max = cluster_df["lat"].max()
idx = (nr_all["dcount"] == 6)
cluster_df["lat"] += abs(cluster_df["lat"].min())
cluster_df["lat"] /= lat_max
cluster_df["lng"] += abs(cluster_df["lng"].min())
cluster_df["lng"] /= lat_max
cluster_df["mbd_lag_5"] /= (cluster_df["mbd_lag_5"].max() * 2)

kmeans = KMeans(20, random_state=22) 
kmeans.fit(cluster_df.loc[idx])

nan_index = np.logical_not(cluster_df.isnull().any(axis=1))

nr_all.loc[nan_index, "cluster_lag_5"] = kmeans.predict(cluster_df.loc[nan_index])

for county in tqdm(counties):
    for j in range(2, 5):
        # Compute lags and rolling mean and std for the cluster label
        # each county was given
        
        nr_all.loc[(nr_all["cfips"] == county) & nan_index, "cluster_lag_" + str(j+4)] = nr_all.loc[(nr_all["cfips"] == county) & nan_index, "cluster_lag_5"].shift(j-1)
        
        s = nr_all.loc[(nr_all["cfips"] == county) & nan_index, "cluster_lag_5"].rolling(j)
        nr_all.loc[(nr_all["cfips"] == county) & nan_index, "cluster_MA_" + str(j)] = s.mean().shift(1)
        nr_all.loc[(nr_all["cfips"] == county) & nan_index, "cluster_Mstd_" + str(j)] = s.std().shift(1)
    
# Separate data into the three different original sets
nr_sub_train = nr_all[nr_all["set"] == "train"].drop("set", axis=1).copy()
nr_valid = nr_all[nr_all["set"] == "valid"].drop("set", axis=1).copy()
nr_test = nr_all[nr_all["set"] == "test"].drop("set", axis=1).copy()

sub_train = sub_train.drop("set", axis=1)
valid = valid.drop("set", axis=1)
sorted_test = sorted_test.drop("set", axis=1)

a


100%|██████████| 3135/3135 [01:24<00:00, 36.98it/s]


## Final model and submission
Fit a CatBoostRegressor to the features we defined. Then, convert the target back to its valid form and prepare data for submission.

In [20]:
nr_final_all = pd.concat([nr_sub_train.dropna(), nr_valid])
nr_final_all = nr_final_all.sort_values("first_day_of_month")
nr_final_all = nr_final_all.drop(columns=["row_id", "county", "state", "active"])

nr_final_test = nr_test.sort_values("first_day_of_month")
nr_final_test_row_id = nr_final_test["row_id"]
nr_final_test = nr_final_test.drop(columns=["row_id", "county", "state", "active"])

nr_preds = pd.DataFrame()

# Define the final model
nr_cat = CatBoostRegressor(loss_function="MAPE",
                           iterations=800,
                           grow_policy='SymmetricTree',
                           verbose=0,
                           learning_rate=0.035,
                           max_depth=6,
                           l2_leaf_reg=0.2,
                           subsample=0.50,
                           max_bin=4096)

# Train the model. We first drop the columns which introduce target leakage or which
# do not help the model at all.
nr_cat.fit(X=nr_final_all.drop(columns=["target", "cfips", "microbusiness_density", "mean_state_tg", "state_tg_std", "first_day_of_month"]),
           y=nr_final_all["target"],
           verbose=0)
# Compute predictions for the test set
nr_final_test["target_preds"] = nr_cat.predict(nr_final_test.drop(columns=["target", "cfips", "microbusiness_density", "mean_state_tg", "state_tg_std", "first_day_of_month"]))

test_dc = sorted(nr_final_test["dcount"].unique())

# Convert "target" to microbusiness density (remember: "target" was the relative
# change in MBD, and we want the MBD per se)

# Note we first perform this operation separately for the first month of the 
# test set. This is because we need the previous month's microbusiness density -
# which is precisely the last month of the training set - to convert the target.

# First month:
for county in counties:
    idx = (nr_final_test["cfips"] == county) & (nr_final_test["dcount"] == test_dc[0])
    nr_final_test.loc[idx , "microbusiness_density"] = (nr_final_test.loc[idx, "target_preds"] + 1) * nr_final_all.loc[(nr_final_all["cfips"] == county) & (nr_final_all["dcount"] == (test_dc[0] - 1)), "microbusiness_density"].values[0]

# Rest of the months:
for dc in test_dc[1:]:
    for county in counties:
        idx = (nr_final_test["cfips"] == county) & (nr_final_test["dcount"] == dc)
        nr_final_test.loc[idx , "microbusiness_density"] = (nr_final_test.loc[idx, "target_preds"] + 1) * nr_final_test.loc[(nr_final_test["cfips"] == county) & (nr_final_test["dcount"] == (dc - 1)), "microbusiness_density"].values[0]

In [31]:
# Save the predictions as a csv to submit them

preds_2022 = pd.concat([revealed["row_id"].reset_index(drop=True), revealed["microbusiness_density"].reset_index(drop=True)], axis=1)
preds_2023 = pd.concat([nr_final_test_row_id.reset_index(drop=True), nr_final_test["microbusiness_density"].reset_index(drop=True)], axis=1)
preds_2023.index = preds_2023.index + preds_2022.shape[0]

subm = pd.concat([preds_2022, preds_2023])
subm.to_csv('submission.csv', index=False)
subm

,row_id,microbusiness_density
0,1001_2022-11-01,3.442677
1,1001_2022-12-01,3.470915
2,1003_2022-11-01,8.257636
3,1003_2022-12-01,8.250630
4,1005_2022-11-01,1.247223
...,...,...
25075,1117_2023-06-01,6.664164
25076,55031_2023-06-01,3.507871
25077,17113_2023-06-01,4.799820
25078,29045_2023-06-01,0.692387
